# Data Retrieval

In this notebook, we shall provide the code required to retrieve the data intended for analysis. It should be noted that appropriate credentials must be supplied in order to access the database.

We shall employ the following modules:

In [ ]:
import os
import subprocess
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

from ipynb_utils import CFG
from ipynb_utils import dump_df

A brief examination of the given database, conducted within a GUI application such as DBeaver, reveals that the following SQL query yields the correct dataset:

In [ ]:
QUERY = """
    SET SCHEMA 'takemehome';
    SELECT * from takemehome.artsy_pageviews;
"""

We load the credentials required for database access from the `.env` file into Python.

In [ ]:
load_dotenv()

DB_CONFIG = {
    "scheme": os.getenv("DB_SCHEME"),
    "database": os.getenv("DATABASE"),
    "user": os.getenv("USER_DB"),
    "password": os.getenv("PASSWORD"),
    "host": os.getenv("HOST"),
    "port": os.getenv("PORT")
}

DB_STRING = (
    "{scheme}://{user}:{password}@{host}:{port}/{database}"
    .format(**DB_CONFIG)
)

/var/folders/sd/w05frkx11tlgfrfknd40sp740000gn/T/ipykernel_9097/3394882986.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pageviews = pd.read_sql(QRY_PAGEVIEWS, conn)
/var/folders/sd/w05frkx11tlgfrfknd40sp740000gn/T/ipykernel_9097/3394882986.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_testuser = pd.read_sql(QRY_TESTUSER, conn)


We establish a connect to the database and load the data specified by the query into a pandas data frame.

In [ ]:
db = create_engine(DB_STRING)

with db.connect() as conn:
    df = pd.read_sql(QUERY, conn)

Let us confirm that the download process has been successful.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   anonymous_id  100000 non-null  object
 1   logged_in     100000 non-null  bool  
 2   received_at   100000 non-null  object
 3   path          100000 non-null  object
 4   referrer      80035 non-null   object
dtypes: bool(1), object(4)
memory usage: 3.1+ MB


In [ ]:
df.sample(10)

,anonymous_id,logged_in,received_at,path,referrer
34689,0383fc2e-c4a5-404a-b306-ae13231fc754,True,2017-03-13 21:52:18.250,/search,https://www.artsy.net/artist/gerard-mannoni
44224,d9e5894a-b097-4880-96ec-8f4637fc507b,False,2017-03-18 00:52:05.873,/article/artsy-editorial-8-female-surrealists-...,https://www.facebook.com/
40426,b3c9e7a3-6e62-4bfe-a5e4-08db0e40612f,False,2017-03-23 23:49:27.677,/artist/isidore-stanislaus-henri-helman,https://www.google.com.br/
57806,04e3ac13-4da1-4192-b0f4-87e40ceda209,False,2017-03-12 03:38:14.907,/the-armory-show-2017,https://www.google.com/
35516,d79263fb-772f-4828-ae25-d24258e418e8,False,2017-03-16 18:15:26.470,/artist/alberto-burri/related-artists,https://www.artsy.net/artist/alberto-burri
99817,887765d9-0936-443c-ad58-227589996564,False,2017-03-25 18:32:22.754,/article/artsy-editorial-7-things-you-didn-t-k...,https://www.facebook.com/
8133,3fbee8c2-a518-4891-8605-a952c6396065,False,2017-03-21 16:40:36.264,/artist/claude-monet,https://www.google.com/
24453,65e20801-6062-491a-b088-f7c6ec8754c5,False,2017-03-19 10:40:04.664,/artwork/kenny-scharf-blob-goy,https://www.google.fr/
8467,2e1c71c9-4d20-4394-abfc-997051bc4e75,False,2017-03-12 00:41:19.905,/artist/frank-stella,https://www.artsy.net/gene/line-form-and-color...
26185,9ff6c341-0d2d-4a2b-80f0-ba3896a6075f,True,2017-03-15 01:06:07.862,/dev/blank,None


The join operation in the SQL query may cause duplication of columns. We shall remove such duplicates immediately.

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]

At last, the data frame is ready to be stored: To preserve the original data structure, we employ a pickle file; the csv version serves solely for direct visual inspection.

In [ ]:
df.to_pickle(CFG["DF_PKL_PATH"])
df.to_csv(CFG["DF_CSV_PATH"], index=False)